In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils



def makedata(images_file,labels_file):
    i=open(images_file,'rb')
    l=open(labels_file,'rb')
    images=list(map(ord, i.read())[16:])
    labels=list(map(ord, l.read())[8:])
    images=torch.tensor(images)
    labels=torch.tensor(labels)
    images=images.reshape(-1,28,28).float()
    i.close()
    l.close()
    return {'images':images,'labels':labels}

class mydataset(Dataset):
    def __init__(self,images_file,labels_file):
        self.data=makedata(images_file,labels_file)
        self.images=self.data['images']
        self.labels=self.data['labels']
    def __len__(self):
        return len(self.labels)
    def __getitem__(self,idx):
        return [self.images[idx],self.labels[idx]]

train_data=mydataset('train-images-idx3-ubyte','train-labels-idx1-ubyte')
len(train_data)
test_data=mydataset('t10k-images-idx3-ubyte','t10k-labels-idx1-ubyte')

batch_size=1000
train_loader=torch.utils.data.DataLoader(dataset=train_data,
                                         batch_size=batch_size,
                                         shuffle=True)

test_loader=torch.utils.data.DataLoader(dataset=test_data,
                                        batch_size=1000,
                                        shuffle=False)

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2)
        self.relu1=nn.Sigmoid()

        self.maxpool1=nn.MaxPool2d(kernel_size=2)

        self.cnn2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2)
        self.relu2=nn.Sigmoid()

        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        self.fc1=nn.Linear(32*7*7,10)
        

    def forward(self,x):
        out=self.cnn1(x)
        out=self.relu1(out)
        out=self.maxpool1(out)
        out=self.cnn2(out)
        out=self.relu2(out)
        out=self.maxpool2(out)
        out=out.view(out.size(0),-1)
        out=self.fc1(out)
        return out


model=CNN()

use_cuda = True

if use_cuda and torch.cuda.is_available():
    model=model.cuda()

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.015)

num_epochs=100
j=0
for _ in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images=images.unsqueeze(1)
        images=Variable(images)
        labels=Variable(labels)
        
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
            
            
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
    total=0
    correct=0
    for im,lab in test_loader:
        im=im.unsqueeze(1)
        im=Variable(im)
        if use_cuda and torch.cuda.is_available():
            im=im.cuda()
            lab=lab.cuda()
        out=model(im)
        _,pred=torch.max(out.data, 1)
        total+=lab.size(0)
        correct+=(pred==lab).sum()
    accuracy=float(correct)/float(total)
    print(loss,accuracy)

(tensor(2.3032, device='cuda:0'), 0.1)
(tensor(2.3025, device='cuda:0'), 0.1)
(tensor(2.3030, device='cuda:0'), 0.1)
(tensor(2.3026, device='cuda:0'), 0.1384)
(tensor(2.3023, device='cuda:0'), 0.1)
(tensor(2.3015, device='cuda:0'), 0.1)
(tensor(2.3030, device='cuda:0'), 0.1)
(tensor(2.3034, device='cuda:0'), 0.1)
(tensor(2.3018, device='cuda:0'), 0.1)
(tensor(2.3011, device='cuda:0'), 0.1)
(tensor(0.8649, device='cuda:0'), 0.6253)
(tensor(0.5752, device='cuda:0'), 0.7699)
(tensor(0.4679, device='cuda:0'), 0.8032)
(tensor(0.4733, device='cuda:0'), 0.8249)
(tensor(0.4333, device='cuda:0'), 0.8325)
(tensor(0.4530, device='cuda:0'), 0.8322)
(tensor(0.4314, device='cuda:0'), 0.837)
(tensor(0.4163, device='cuda:0'), 0.8348)
(tensor(0.4292, device='cuda:0'), 0.8503)
(tensor(0.3675, device='cuda:0'), 0.844)
(tensor(0.3920, device='cuda:0'), 0.8454)
(tensor(0.3636, device='cuda:0'), 0.8477)
(tensor(0.3895, device='cuda:0'), 0.8529)
(tensor(0.3956, device='cuda:0'), 0.8485)
(tensor(0.3592, devic